## Objetivo do notebook
#### O objetivo deste notebook é realizar a leitura de dados das origens do dataset avaliações de turmas e gravar de forma versionada na camada bronze


### Import Libs

In [1]:
!pip install minio

In [2]:
from pyspark.sql import SparkSession
from datetime import datetime
from minio import Minio
from minio.error import S3Error
from pyspark.sql.functions import lit
from pyspark.sql.functions import current_timestamp

import os
import pandas as pd
import glob

### Definição de variáveis

In [3]:
# Sessão Spark
spark = SparkSession.builder.getOrCreate()

# Parametros de input e output das origens
camadaEscrita = 'bronze'
pasta = 'avaliacoes'
url = "https://raw.githubusercontent.com/willianmaria/00006-2024-data/main/turma_avaliacoes.csv"
temp_blobs = '/home/jovyan/notebooks/temporary_blobs/' # pasta temporária para armazenamento de objetos

# Conexão ao miniIO
minio_endpoint = 'minio:9000'
minio_access_key = 'minioaccesskey'
minio_secret_key = 'miniosecretkey'
minio_object_name = pasta
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

# Definição de variáveis para versionamento dos dados na camada bronze
timestamp = datetime.today().strftime('%Y%m%d')
minio_path = f'{pasta} / {pasta + timestamp}'

### Leitura e Tratamento dos dados

In [4]:
# leitura do CSV
df_avaliacoes = pd.read_csv(url)
#Geração de um dataframe spark
df_avaliacoes = spark.createDataFrame(df_avaliacoes)
#Criação de colunas de etl_Date
df_avaliacoes = df_avaliacoes.withColumn("etl_date", current_timestamp())

### Gravação do dataframe em um diretório temporário

In [5]:
df_avaliacoes.write.parquet(temp_blobs, mode="overwrite")

### Gravação na camada Bronze

In [6]:
# Recria bucket caso não exista
if not minio_client.bucket_exists(camadaEscrita):
    minio_client.make_bucket(camadaEscrita)

# Lista todos os arquivos Parquet no diretório temporário de blobs
arquivos_parquet = glob.glob(os.path.join(temp_blobs, '*.parquet'))

# Itera sobre a lista de arquivos Parquet e envia cada um para o MinIO
for arquivo_parquet in arquivos_parquet:
    try:
        # Envia o arquivo para a camada bronze
        nome_arquivo = os.path.basename(arquivo_parquet)
        minio_client.fput_object(camadaEscrita, os.path.join(minio_path, nome_arquivo), arquivo_parquet)
        print(f"Arquivo '{nome_arquivo}' enviado com sucesso para o MinIO em '{os.path.join(minio_path, nome_arquivo)}'.")
    except S3Error as e:
        print(f"Erro ao enviar o arquivo para o MinIO: {e}")


Arquivo 'part-00000-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes / avaliacoes20240212/part-00000-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet'.
Arquivo 'part-00001-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes / avaliacoes20240212/part-00001-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet'.
Arquivo 'part-00002-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes / avaliacoes20240212/part-00002-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet'.
Arquivo 'part-00003-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes / avaliacoes20240212/part-00003-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet'.
Arquivo 'part-00004-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet' enviado com sucesso para o MinIO em 'avaliacoes / aval

### Remove dados do diretório temporário de blobs

In [7]:
# Lista todos os arquivos na pasta
arquivos_na_pasta = os.listdir(temp_blobs)

# Itera sobre os arquivos e os deleta
for arquivo in arquivos_na_pasta:
    caminho_completo = os.path.join(temp_blobs, arquivo)
    try:
        if os.path.isfile(caminho_completo):
            os.remove(caminho_completo)
            print(f'{caminho_completo} deletado com sucesso.')
    except Exception as e:
        print(f'Erro ao deletar {caminho_completo}: {e}')

spark.stop()

/home/jovyan/notebooks/temporary_blobs/.part-00000-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00001-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00002-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00003-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.


/home/jovyan/notebooks/temporary_blobs/.part-00004-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00005-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00006-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00007-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00008-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00009-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00010-de534d58-88bd-4695-bf7f-6d31ba313d1a-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/temporary_blobs/.part-00011-de534d58-88